In [ ]:
import pandas as pd
from constants import distinct_samples
from SALib.sample.sobol import sample

from mesa.examples.advanced.epstein_civil_violence.agents import (
    CitizenState,
)

# Generate points

In [ ]:
param_names = ["legitimacy", "active_threshold"]
param_bounds = [[0.5, 1.0], [0.01, 0.5]]

problem = {
    "num_vars": len(param_names),
    "names": param_names,
    "bounds": param_bounds,
}

model_reporters = {
    "active": CitizenState.ACTIVE.name,
    "quiet": CitizenState.QUIET.name,
    "arrested": CitizenState.ARRESTED.name,
}

params = sample(problem, distinct_samples)
data = pd.DataFrame(params, columns=param_names)

data.to_csv("inputs/data.csv")

print(data)

    legitimacy  active_threshold
0     0.983783          0.276637
1     0.718424          0.276637
2     0.983783          0.296700
3     0.983783          0.296700
4     0.718424          0.276637
5     0.718424          0.296700
6     0.654459          0.034525
7     0.801492          0.034525
8     0.654459          0.144335
9     0.654459          0.144335
10    0.801492          0.034525
11    0.801492          0.144335
12    0.515733          0.406331
13    0.559163          0.406331
14    0.515733          0.388539
15    0.515733          0.388539
16    0.559163          0.406331
17    0.559163          0.388539
18    0.845040          0.148873
19    0.960470          0.148873
20    0.845040          0.052376
21    0.845040          0.052376
22    0.960470          0.148873
23    0.960470          0.052376
24    0.784504          0.462347
25    0.863363          0.462347
26    0.784504          0.100179
27    0.784504          0.100179
28    0.863363          0.462347
29    0.86